# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# Create our features
features = df.drop(target, axis=1)
X = pd.get_dummies(features)

# Create our target
y = df[target]

In [6]:
# X.shape()
# X.describe()

In [7]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, stratify=y)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classification report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [9]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=20)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [10]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
logisticReg = LogisticRegression(solver='lbfgs', random_state=20)
logisticReg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=20)

In [11]:
# Calculated the balanced accuracy score
y_pred = logisticReg.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
ros_bas = balanced_accuracy_score(y_test, y_pred)
ros_bas


0.6302843145549553

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Visual
ros_cm = pd.DataFrame(
    cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])

ros_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,48,39
Actual Low_Risk,4984,12134


In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
ros_ICM = classification_report_imbalanced(y_test, y_pred)
print(ros_ICM)


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.55      0.71      0.02      0.63      0.38        87
   low_risk       1.00      0.71      0.55      0.83      0.63      0.40     17118

avg / total       0.99      0.71      0.55      0.82      0.63      0.40     17205



### SMOTE Oversampling

In [14]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=20)
X_resampled, y_resampled = SMOTE(random_state=20).fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [15]:
# Train the Logistic Regression model using the resampled data
logisticReg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=20)

In [16]:
# Calculated the balanced accuracy score
y_pred = logisticReg.predict(X_test)
smote_bas = balanced_accuracy_score(y_test, y_pred)
smote_bas

0.6343373178465096

In [17]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
smote_cm = pd.DataFrame(
    cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])

smote_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,49,38
Actual Low_Risk,5042,12076


In [18]:
# Print the imbalanced classification report
smote_ICM = classification_report_imbalanced(y_test, y_pred)
print(smote_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.56      0.71      0.02      0.63      0.39        87
   low_risk       1.00      0.71      0.56      0.83      0.63      0.40     17118

avg / total       0.99      0.70      0.56      0.82      0.63      0.40     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [19]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=20)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [20]:
# Train the Logistic Regression model using the resampled data
logisticReg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=20)

In [21]:
# Calculated the balanced accuracy score
y_pred = logisticReg.predict(X_test)
cluster_bas = balanced_accuracy_score(y_test, y_pred)
cluster_bas

0.499463158361233

In [22]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cluster_cm = pd.DataFrame(
    cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])

cluster_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,43,44
Actual Low_Risk,8479,8639


In [23]:
# Print the imbalanced classification report
cluster_ICM = classification_report_imbalanced(y_test, y_pred)
print(cluster_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.49      0.50      0.01      0.50      0.25        87
   low_risk       0.99      0.50      0.49      0.67      0.50      0.25     17118

avg / total       0.99      0.50      0.49      0.67      0.50      0.25     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [24]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state=20)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [25]:
# Train the Logistic Regression model using the resampled data
logisticReg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=20)

In [26]:
# Calculated the balanced accuracy score
y_pred = logisticReg.predict(X_test)
smoteenn_bas = balanced_accuracy_score(y_test, y_pred)
smoteenn_bas

0.6264740482895601

In [27]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
smoteenn_cm = pd.DataFrame(
    cm, index=["Actual High_Risk", "Actual Low_Risk"], columns=["Predicted High_Risk", "Predicted Low_Risk"])

smoteenn_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,54,33
Actual Low_Risk,6295,10823


In [28]:
# Print the imbalanced classification report
smoteenn_ICM = classification_report_imbalanced(y_test, y_pred)
print(smoteenn_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.63      0.02      0.63      0.39        87
   low_risk       1.00      0.63      0.62      0.77      0.63      0.39     17118

avg / total       0.99      0.63      0.62      0.77      0.63      0.39     17205



# Results for Analysis
---
## Random Oversampling

In [38]:
print(f"Balanced Accuracy Score: {round(ros_bas*100,2):.1f}%")

Balanced Accuracy Score: 63.0%


In [37]:
ros_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,48,39
Actual Low_Risk,4984,12134


In [39]:
print(ros_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.55      0.71      0.02      0.63      0.38        87
   low_risk       1.00      0.71      0.55      0.83      0.63      0.40     17118

avg / total       0.99      0.71      0.55      0.82      0.63      0.40     17205



---
## SMOTE

In [40]:
print(f"Balanced Accuracy Score: {round(smote_bas*100,2):.1f}%")

Balanced Accuracy Score: 63.4%


In [41]:
smote_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,49,38
Actual Low_Risk,5042,12076


In [43]:
print(smote_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.56      0.71      0.02      0.63      0.39        87
   low_risk       1.00      0.71      0.56      0.83      0.63      0.40     17118

avg / total       0.99      0.70      0.56      0.82      0.63      0.40     17205



---
## Cluster Centroids

In [44]:
print(f"Balanced Accuracy Score: {round(cluster_bas*100,2):.1f}%")

Balanced Accuracy Score: 50.0%


In [45]:
cluster_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,43,44
Actual Low_Risk,8479,8639


In [46]:
print(cluster_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.49      0.50      0.01      0.50      0.25        87
   low_risk       0.99      0.50      0.49      0.67      0.50      0.25     17118

avg / total       0.99      0.50      0.49      0.67      0.50      0.25     17205



---
## SMOTEENN

In [47]:
print(f"Balanced Accuracy Score: {round(smoteenn_bas*100,2):.1f}%")

Balanced Accuracy Score: 62.6%


In [48]:
smoteenn_cm

,Predicted High_Risk,Predicted Low_Risk
Actual High_Risk,54,33
Actual Low_Risk,6295,10823


In [49]:
print(smoteenn_ICM)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.63      0.02      0.63      0.39        87
   low_risk       1.00      0.63      0.62      0.77      0.63      0.39     17118

avg / total       0.99      0.63      0.62      0.77      0.63      0.39     17205

